<a href="https://colab.research.google.com/github/jolly-thomas/cuddly-lamp/blob/main/Supervised%20Learning/Gini_Index_Implementation_From_Scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [21]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

In [36]:
# For now this code works only for categorical datasets . For numerical datasets we can categorize data using k-bins, k-means , or simply use
# quaniles or percentiles.

data = {
    "PetalColor": ["Red", "Yellow", "Pink", "Red", "White", "Pink", "Yellow", "White", "Red", "Pink",
                   "Purple", "Orange", "Blue", "Red", "Yellow", "Pink", "White", "White", "Red", "Pink",
                   "Yellow", "Orange", "White", "Red", "Blue"],
    "LeafSize": ["Small", "Large", "Medium", "Large", "Small", "Medium", "Small", "Large", "Medium", "Small",
                 "Large", "Small", "Medium", "Large", "Small", "Medium", "Medium", "Small", "Large", "Small",
                 "Large", "Medium", "Small", "Large", "Medium"],
    "StemLength": ["Short", "Long", "Short", "Medium", "Medium", "Short", "Long", "Short", "Long", "Medium",
                   "Medium", "Short", "Long", "Short", "Medium", "Long", "Medium", "Short", "Medium", "Long",
                   "Short", "Medium", "Long", "Short", "Medium"],
    "Fragrance": ["Strong", "Mild", "None", "Strong", "Mild", "Strong", "None", "Mild", "Strong", "Mild",
                  "None", "Strong", "Mild", "Strong", "Mild", "None", "Mild", "Strong", "None", "Mild",
                  "Strong", "Mild", "None", "Strong", "Mild"],
    "Season": ["Spring", "Summer", "Spring", "Winter", "Autumn", "Spring", "Summer", "Winter", "Autumn", "Spring",
               "Winter", "Spring", "Autumn", "Winter", "Summer", "Spring", "Autumn", "Spring", "Winter", "Summer",
               "Spring", "Autumn", "Winter", "Summer", "Spring"],
    "FlowerType": ["Rose", "Sunflower", "Tulip", "Rose", "Lily", "Tulip", "Sunflower", "Lily", "Rose", "Tulip",
                   "Orchid", "Daisy", "Hydrangea", "Rose", "Sunflower", "Tulip", "Lily", "Lily", "Rose", "Tulip",
                   "Sunflower", "Daisy", "Lily", "Rose", "Hydrangea"]
}
df = pd.DataFrame(data)
df

,PetalColor,LeafSize,StemLength,Fragrance,Season,FlowerType
0,Red,Small,Short,Strong,Spring,Rose
1,Yellow,Large,Long,Mild,Summer,Sunflower
2,Pink,Medium,Short,None,Spring,Tulip
3,Red,Large,Medium,Strong,Winter,Rose
4,White,Small,Medium,Mild,Autumn,Lily
5,Pink,Medium,Short,Strong,Spring,Tulip
6,Yellow,Small,Long,None,Summer,Sunflower
7,White,Large,Short,Mild,Winter,Lily
8,Red,Medium,Long,Strong,Autumn,Rose
9,Pink,Small,Medium,Mild,Spring,Tulip


In [37]:
x_train,x_test,y_train,y_test=train_test_split(df.drop(columns=['FlowerType']),df["FlowerType"],test_size=0.2)

In [38]:
# Calculate Class Probabilities for attributes of given data x for class labels y

def class_prob(x,y):
  x_labels,x_count=np.unique(x,return_counts=True)
  prob={}
  for i in range(len(x_labels)):
    filtered=y[x==x_labels[i]]
    num,num_count=np.unique(filtered,return_counts=True)
    prob[x_labels[i]]=(num_count/x_count[i])
  return prob

In [39]:
# Calculate gini index using the class probability for every feature attribute

def gini_index(class_prob):
  for i in class_prob:
    gini_index=1-np.sum(np.square(class_prob[i]))
    class_prob[i]=gini_index
  return class_prob

In [40]:
# Calculate the weighted average of gini indexes for the whole feature

def weighted_avg(gini,x):
  x_labels,x_count=np.unique(x,return_counts=True)
  wa=0
  for i in range(len(x_labels)):
    prob=x_count[i]/sum(x_count)
    wa+=gini[x_labels[i]]*prob
  return wa

In [41]:
# Compare all the weighted averages and choose the best split

def best_split(data,target_data):
  gini={}
  for column in data.columns:
    column_data=np.array(data[column])
    gini[column]=weighted_avg(gini_index(class_prob(column_data,target_data)),column_data)
  min_val=min(gini,key=gini.get)
  return min_val


In [42]:
# Define a Node Class so that we can create a tree

class Node:
  def __init__(self,data,label):
    self.data = data
    self.label=label
    self.child =[]
  def add_child(self,child):
    self.child.append(child)

# Initiate an empty node to keep track of the tree

start=Node(data=None,label=None)

In [43]:
# Function for creating the tree
# head is the initial node
# level is the depth of the tree (used for printing the tree)


def decision_tree(data,target,head,level):
  if data.empty or target.empty:
    return None
  best_split_value=best_split(data,target)  # evaluate best split
  if not best_split_value:
    return None
  print("---"*level+f"{best_split_value}")
  head.data=best_split_value                # set head value to the best split feature
  label=np.unique(data[best_split_value])   # label contains attributes of best split feature
  for i in label:
    filtered_data=target[data[best_split_value]==i]     # create a dataset from the target where only chosen label(attribute) exists
    unique,count=np.unique(filtered_data,return_counts=True)      # count unique class labels (target)
    hash=dict(zip(unique,count))
    y_max=max(hash,key=hash.get)                        # Get the count of the class label with max occurences
    prob=hash[y_max]/len(filtered_data)                 # Calculate probability of the max class label
    if prob >= 0.95:                                    # I have used 0.95 as the threshold you can use 1 or any other threshold to prevent overfitting & support pruning
      head.add_child(Node(data=y_max,label=i))          # If probabilty is greater than threshold, it means it is a leaf node (decision is found)
      print("---"*level +i+"---"+ y_max)                # create a leaf node and add it to the child of head node
    else:
      print("---"*level +i)                             # If decision is not found then we can further splilt the data using all the steps done before
      empty_node=Node(data=None,label=i)                # drop the best split column because it is already used & only use the data where label (i) exists
      head.add_child(decision_tree(data[data[best_split_value]==i].drop(columns=[best_split_value]),target[data[best_split_value]==i],empty_node,level+1))
  return head                                           # recursively implement and return head


In [44]:
tree=decision_tree(x_train,y_train,start,0)

PetalColor
Blue---Hydrangea
Orange---Daisy
Pink---Tulip
Purple---Orchid
Red---Rose
White---Lily
Yellow---Sunflower


In [45]:
# Traverse function used to traverse throught the tree

def traverse(data,tree):
  if not tree.child and tree.data != None:
        return tree.data
  for i in tree.child:
    if data[tree.data]==i.label:
      return traverse(data,i)

In [46]:
# predict the class labels
# tree is the head of the decision tree returned by the decision tree function
def predict(data,tree):
  y=[]
  for i in range(data.shape[0]):
    y.append(traverse(data.iloc[i],tree))
  return y

In [47]:
predict(x_test,tree)

['Rose', 'Lily', 'Rose', 'Tulip', 'Lily']

In [48]:
y_test

,FlowerType
8,Rose
17,Lily
3,Rose
2,Tulip
4,Lily
